In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./data/agnews/train.csv')

In [ ]:
df

In [ ]:
# show the first 10 titles and descriptions in the dataset full text
for i in range(3):
    title = df['Title'][i]
    description = df['Description'][i]
    label = df['Class Index'][i]
    num_tokens = description.count(' ') + 1
    print('Title: ', title)
    print('Description: ', description)
    print('Number of tokens: ', num_tokens)
    print('Label: ', label)

    print('===========================================')

In [ ]:
tokens = []
for i in range(len(df)):    
    description = df['Description'][i]    
    num_tokens = description.count(' ') + 1
    tokens.append(num_tokens)
df['tokens'] = tokens

In [ ]:
df.describe()

In [ ]:

df = df[['Class Index', 'Description']]

In [ ]:
df

In [ ]:
train_df = pd.read_csv('./data/agnews/train.csv')
test_df = pd.read_csv('./data/agnews/test.csv')

train_df = train_df[['Class Index', 'Description']]
test_df = test_df[['Class Index', 'Description']]



In [ ]:
import numpy as np

np.savetxt('./data/agnews/train.txt', train_df.values, fmt='%s', delimiter='\t')
np.savetxt('./data/agnews/test.txt', test_df.values, fmt='%s', delimiter='\t')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
kaggle_med = pd.read_csv('./data/original/kaggle_med/train.txt',delimiter='\t', header=None, names=['class', 'text'])
kaggle_med['text'] = kaggle_med['class'].apply(lambda x: x.split(' ',1)[1]) 
kaggle_med['class'] = kaggle_med['class'].apply(lambda x: x.split(' ',1)[0])

In [ ]:
kaggle_med['class'].unique()

In [ ]:
kaggle_med.describe()

In [ ]:
kaggle_med['class'].value_counts()

In [ ]:
kaggle_med.text.loc[kaggle_med['class'] == '7.']

In [ ]:
kaggle_med.text.loc[kaggle_med['class'] == '6.']

In [ ]:
kaggle_med.text.loc[kaggle_med['class'] == '0']

In [ ]:
kaggle_med.text.loc[kaggle_med['class'] == 'and']

In [ ]:
new_df_train = kaggle_med.drop(kaggle_med[(kaggle_med['class'] == 'and') | (kaggle_med['class'] == '6.') | (kaggle_med['class'] == '7.')].index)


In [ ]:
new_df_train.describe()

In [ ]:
np.savetxt('./data/original/kaggle_med/train.txt', new_df_train.values, fmt='%s', delimiter='\t')

In [ ]:
kaggle_med = pd.read_csv('./data/original/kaggle_med/test.txt',delimiter='\t', header=None, names=['class', 'text'])
kaggle_med['text'] = kaggle_med['class'].apply(lambda x: x.split(' ',1)[1]) 
kaggle_med['class'] = kaggle_med['class'].apply(lambda x: x.split(' ',1)[0])

In [ ]:
kaggle_med['class'].unique()

In [ ]:
kaggle_med.describe()

In [ ]:
df = pd.read_csv('./data/original/kaggle_med/train.txt',delimiter='\t', header=None, names=['class', 'text'])

In [ ]:
df

In [ ]:
duplicates = df[df.duplicated()]
len(duplicates)

In [ ]:
print(duplicates)

In [ ]:
duplicates = df[df['text'].duplicated()]
len(duplicates)

In [ ]:
for i in range(len(duplicates)):
    print(duplicates['text'].iloc[i])
    print('===========================================')

In [ ]:
duplicates = df.groupby('text').apply(lambda x: pd.Series({
    'class': list(x['class']),
    'count': len(x)
})).reset_index()
duplicates = duplicates[duplicates.duplicated(subset='class')]
for i in range(len(duplicates)):
    count = duplicates['count'].iloc[i]
    if count > 2:
        print('text: ',duplicates['text'].iloc[i])
        print('class: ',duplicates['class'].iloc[i])
        print('count: ',duplicates['count'].iloc[i])
        print('===========================================')

In [ ]:
df = pd.read_csv('./data/original/cardio/train.txt',delimiter='\t', header=None, names=['class', 'text'])
duplicates = df[df.duplicated()]
len(duplicates)

## new session
### augmentation

In [ ]:
import numpy as np
import pandas as pd
import textattack
from functions import *

In [ ]:
# Define the input paths for the datasets
dict_datasets = {
    'agnews': './data/original/agnews/train.txt',
    'cardio': './data/original/cardio/train.txt',
    'cr': './data/original/cr/train.txt',
    'pc': './data/original/pc/train.txt',
    'pc_clean': './data/original/pc/train_clean.txt',
    'sst2': './data/original/sst2/train.txt',
    'subj': './data/original/subj/train.txt',
    'trec': './data/original/trec/train.txt',
    'yelp': './data/original/yelp/train.txt'
}

# Define the augmentation methods and their names
dict_methods = {
    'aeda': 'aeda_augmenter',
    'backtranslation': 'backtranslation_augmenter',
    'checklist': 'checklist_augmenter',
    'clare': 'clare_augmenter',
    'eda': 'eda_augmenter',    
    'wordnet': 'wordnet_augmenter'
}


In [ ]:
# import snoop
# @snoop(depth=2)
def aug_samples(dataset_name, method_names, fraction, pct_words_to_swap, transformations_per_example, include_original):
    data = load_data(dict_datasets[dataset_name])
    print(f'Loaded {len(data)} rows of data from {dict_datasets[dataset_name]}')
    #data = data.sample(frac=fraction)
    #print(f'Sampled {len(data)} rows with fraction={fraction}')
    for method in method_names:
        if method not in dict_methods:
            raise ValueError(f"Method '{method}' is not supported")
        augmented_data = augment_text(
            data,
            dict_methods[method],
            fraction=fraction,
            pct_words_to_swap=pct_words_to_swap,
            transformations_per_example=transformations_per_example,
            label_column='class',
            target_column='text',
            include_original=include_original
        )
        augmented_data = augmented_data[['class', 'text']]
        np.savetxt(
            f'./data/augmented/{dataset_name}/{fraction}_{method}_{transformations_per_example}.txt',
            augmented_data.values,
            fmt='%s',
            delimiter='\t'
        )
        print(f'number of augmented samples: {len(augmented_data)}')

In [ ]:
aug_samples('agnews', ['eda'], 0.001, 0.5, 2, False)

In [ ]:
def aug_samples(dataset_name, method_names, fraction, pct_words_to_swap, transformations_per_example, include_original):
    data = load_data(dict_datasets[dataset_name])
    methods = method_names
    for method in methods:
        augmented_data = augment_text( data, method, fraction=fraction, pct_words_to_swap=pct_words_to_swap, transformations_per_example=transformations_per_example,
                                                            label_column='class', target_column='text', include_original=include_original)
        augmented_data = augmented_data[['class','text']]
        np.savetxt(f'./data/augmented/{dataset_name}/{fraction}_{method}_{transformations_per_example}.txt', augmented_data.values, fmt='%s', delimiter='\t')

In [ ]:
aug_samples('pc', ['checklist_augmenter'], 0.001, 0.5, 2, True)

# test

In [ ]:
data = load_data(dict_datasets['pc'])
df = data.copy( )

In [ ]:
df= df[~(df['text'].isin(['','none','nan','null','NaN','NULL','Null','None','NONE']))].reset_index(drop=True)

In [ ]:
np.savetxt('./data/original/pc/train_clean.txt', df.values, fmt='%s', delimiter='\t')

In [ ]:
aug_samples('pc_clean', ['checklist'], 0.001, 0.5, 2, True)

In [ ]:
df = pd.read_csv('./data/original/yelp/yelp.csv')
df = df[['stars', 'text']]
for i in range(len(df)):
    df.loc[i, "text"] = re.sub(r"[\\']", "", df.loc[i, "text"])

In [ ]:
import re

def clean_text(text):    
    text = re.sub(r"\n+", " ", text)
    text = re.sub(r"\t+", " ", text)
    text = re.sub(r"\r+", " ", text)
    text = re.sub(r"/\sThe+", "The", text)
    text = re.sub(r"[\\']+", "'", text)
    
    return text

In [ ]:
from sklearn.model_selection import train_test_split
import re
train, test = train_test_split(df, test_size=0.1, random_state=42)
train.to_csv('./data/original/yelp/train.csv', index=False)
test.to_csv('./data/original/yelp/test.csv', index=False)

test['text'] = test['text'].apply(lambda x: clean_text(x))
train['text'] = train['text'].apply(lambda x: clean_text(x))



np.savetxt('./data/original/yelp/train.txt', train.values, fmt='%s', delimiter='\t')
np.savetxt('./data/original/yelp/test.txt', test.values, fmt='%s', delimiter='\t')

In [ ]:
pd.read_csv('./data/original/yelp/train.txt',delimiter='\t', header=None, names=['class', 'text'])

## new session
### augmentation

In [ ]:
# Define the input paths for the datasets
dict_datasets = {
    'agnews': './data/original/agnews/train.txt',
    'cardio': './data/original/cardio/train.txt',
    'cr': './data/original/cr/train.txt',
    'pc': './data/original/pc/train.txt',
    'pc_clean': './data/original/pc/train_clean.txt',
    'sst2': './data/original/sst2/train.txt',
    'subj': './data/original/subj/train.txt',
    'trec': './data/original/trec/train.txt',
    'kaggle_med': './data/original/kaggle_med/train.txt',
    'yelp': './data/original/yelp/train.txt',
    'bbc': './data/original/bbc/train.txt'
}

# Define the augmentation methods and their names
dict_methods = {
    'aeda': 'aeda_augmenter',
    'backtranslation': 'backtranslation_augmenter',
    'checklist': 'checklist_augmenter',
    'clare': 'clare_augmenter',
    'eda': 'eda_augmenter',
    'embedding': 'embedding_augmenter',
    'deletion': 'deletion_augmenter',
    'wordnet': 'wordnet_augmenter'
}

In [ ]:
def aug_samples(dataset_name, method_names, fraction, pct_words_to_swap, transformations_per_example, include_original):
    path = dict_datasets[dataset_name]
    data = pd.read_csv(path, delimiter='\t', header=None, names=['class', 'text'])
    data = data.sample(frac=fraction)
    print(f'Loaded {len(data)} rows of data from {dict_datasets[dataset_name]}')
    #data = data.sample(frac=fraction)
    #print(f'Sampled {len(data)} rows with fraction={fraction}')
    for method in method_names:
        if method not in dict_methods:
            raise ValueError(f"Method '{method}' is not supported")
        augmented_data = augment_text(
            data,
            dict_methods[method],
            fraction=1,
            pct_words_to_swap=pct_words_to_swap,
            transformations_per_example=transformations_per_example,
            label_column='class',
            target_column='text',
            include_original=include_original
        )
        augmented_data = augmented_data[['class', 'text']]
        np.savetxt(
            f'./data/augmented/{dataset_name}/{fraction}_{method}_{transformations_per_example}.txt',
            augmented_data.values,
            fmt='%s',
            delimiter='\t'
        )
        print(f'number of augmented samples: {len(augmented_data)}')

In [ ]:
aug_samples('yelp', ['eda'], 0.001, 0.5, 2, False)

In [ ]:
df = pd.read_csv('./data/original/yelp/train.txt',delimiter='\t', header=None, names=['class', 'text'])

In [ ]:
df.iloc[4284,1]

In [ ]:
import numpy as np
import pandas as pd
import textattack
from functions1 import *
method_names = ['eda']
dataset_name = 'yelp'
fraction = 0.001
path = dict_datasets[dataset_name]
data = pd.read_csv(path, delimiter='\t', header=None, names=['class', 'text'])
data = data.sample(frac=fraction)
print(f'Loaded {len(data)} rows of data from {dict_datasets[dataset_name]}')

In [ ]:
# Define the input paths for the datasets
dict_datasets = {
    'agnews': './data/original/agnews/train.txt',
    'cardio': './data/original/cardio/train.txt',
    'cr': './data/original/cr/train.txt',
    'pc': './data/original/pc/train.txt',
    'pc_clean': './data/original/pc/train_clean.txt',
    'sst2': './data/original/sst2/train.txt',
    'subj': './data/original/subj/train.txt',
    'trec': './data/original/trec/train.txt',
    'kaggle_med': './data/original/kaggle_med/train.txt',
    'yelp': './data/original/yelp/train.txt'
}

# Define the augmentation methods and their names
dict_methods = {
    'aeda': 'aeda_augmenter',
    'backtranslation': 'backtranslation_augmenter',
    'checklist': 'checklist_augmenter',
    'clare': 'clare_augmenter',
    'eda': 'eda_augmenter',
    'embedding': 'embedding_augmenter',
    'deletion': 'deletion_augmenter',
    'wordnet': 'wordnet_augmenter'
}

import numpy as np
import pandas as pd
import textattack
from functions import *
def aug_samples(name,frac):
    path = dict_datasets[name]
    data = load_data(path)
    data = data.sample(frac=frac)
    print(f'Loaded {len(data)} rows of data from {name}')
    #methods = ['eda_augmenter','wordnet_augmenter','aeda_augmenter','backtranslation_augmenter']
    methods = ['checklist_augmenter']
    for method in methods:
        augmented_data = augment_text(data, method,fraction=1,pct_words_to_swap=0.2 ,transformations_per_example=1,
                    label_column='class',target_column='text',include_original=True)
        augmented_data = augmented_data[['class','text']]
        np.savetxt(f'./data/augmented/{name}/{method}_aug.txt', augmented_data.values, fmt='%s', delimiter='\t')

In [ ]:
aug_samples('pc',0.001)

In [ ]:
import numpy as np
import pandas as pd
import textattack
import logging
from functions import *

# Define the input paths for the datasets
dict_datasets = {
    'agnews': './data/original/agnews/train.txt',
    'cardio': './data/original/cardio/train.txt',
    'cr': './data/original/cr/train.txt',
    'pc': './data/original/pc/train.txt',
    'pc_clean': './data/original/pc/train_clean.txt',
    'sst2': './data/original/sst2/train.txt',
    'subj': './data/original/subj/train.txt',
    'trec': './data/original/trec/train.txt',
    'kaggle_med': './data/original/kaggle_med/train.txt',
    'yelp': './data/original/yelp/train.txt'
}

# Define the augmentation methods and their names
dict_methods = {
    'aeda': 'aeda_augmenter',
    'backtranslation': 'backtranslation_augmenter',
    'checklist': 'checklist_augmenter',
    'clare': 'clare_augmenter',
    'eda': 'eda_augmenter',
    'wordnet': 'wordnet_augmenter'
}

aug_num = [1,2,4]
aug_frac = [0.1,0.2,0.5,1]

logging.basicConfig(filename='augment.log', filemode='a', format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)

def aug_samples(name, frac):
    path = dict_datasets[name]
    data = load_data(path)
    try:
        data = data.sample(frac=frac)
    except:
        data = data.sample(n=1)
    logging.info(f'Loaded {len(data)} rows of data from {name}')
    
    #methods = ['eda_augmenter','wordnet_augmenter','aeda_augmenter','backtranslation_augmenter']
    methods = ['checklist_augmenter', 'eda_augmenter', 'wordnet_augmenter', 'aeda_augmenter', 'backtranslation_augmenter', 'clare_augmenter']
    
    for method in methods:
        augmented_data = augment_text(data, method, fraction=1, pct_words_to_swap=0.2, transformations_per_example=1,
                                      label_column='class', target_column='text', include_original=True)
        augmented_data = augmented_data[['class','text']]
        np.savetxt(f'./data/augmented/{name}/{method}_aug.txt', augmented_data.values, fmt='%s', delimiter='\t')
        logging.info(f'Augmented {len(augmented_data)} rows of data from {name} using {method} method')


In [ ]:
aug_samples('yelp', 0.0001)

In [ ]:
#path = dict_datasets['yelp']
path = './data/original/yelp/train.csv'
df = load_data(path)
#df = pd.read_csv(path,  names=['text'])

In [ ]:
df 

In [ ]:
df['class'] = df['text'].apply(lambda x: x.split('\t')[0])

In [ ]:
df

In [ ]:
df['text'].iloc[1]

In [ ]:
df['text'].iloc[0]


In [ ]:
for i in range(10):
    print(df['text'].iloc[i])

In [ ]:
df

In [ ]:
import pandas as pd

def load_data(path):
    """
    Loads data from a txt file.
    """
    # check file format
    if path.endswith('.txt'):
        try:
            df = pd.read_csv(path, sep='|', header=None, names=['text'])
        except:
            df = pd.read_csv(path, sep='\t', header=None, names=['text'])
        df['class'], df['text'] = zip(*df['text'].apply(lambda x: x.split('\t', 1)))
    else:
        df = pd.read_csv(path, header=None, names=['class', 'text'])

    return df[['class', 'text']]


In [ ]:
path = './data/original/yelp/train.csv'
df = load_data(path)

In [ ]:
df

In [ ]:
import pandas as pd

def load_data(path):
    """
    Loads data from a txt file.
    """
    # check file format
    if path.endswith('.txt'):
        try:
            df = pd.read_csv(path, sep='|', header=None, names=['text'])
        except:
            try:
                df = pd.read_csv(path, sep='\t', header=None, names=['text'])
            except:
                df = pd.read_csv(path, header=None, names=['text'])
                df['class'] = df['text'].apply(lambda x: x.split(' ',1)[0])
                df['text'] = df['text'].apply(lambda x: x.split(' ',1)[1])
        else:
            df['class'], df['text'] = zip(*df['text'].apply(lambda x: x.split('\t', 1)))
    else:
        df = pd.read_csv(path, header=None, names=['class', 'text'],skiprows=1)

    return df[['class', 'text']]


In [ ]:
path = './data/original/bbc/train.csv'
df = load_data(path)

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
import re
import pandas as pd
import numpy as np

In [ ]:
# bbc = pd.read_csv('./data/original/bbc/bbc.csv', header=None, names=['class', 'text'],skiprows=1)
# bbc['class'] = bbc['class'].map({'business': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'tech': 4})
# train, test = train_test_split(bbc, test_size=0.1, random_state=42)

# train.to_csv('./data/original/bbc/train.csv', index=False)
# test.to_csv('./data/original/bbc/test.csv', index=False)

# np.savetxt('./data/original/bbc/train.txt', train.values, fmt='%s', delimiter='\t')
# np.savetxt('./data/original/bbc/test.txt', test.values, fmt='%s', delimiter='\t')

In [ ]:
test = pd.read_csv('./data/original/bbc/test.csv')

In [ ]:
test

In [ ]:
train_df['class'] = train_df['Category'].map({'business': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'tech': 4})
test_df['class'] = test_df['Category'].map({'business': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'tech': 4})

# train_df = train_df[['class', 'Text']]
# test_df = test_df[['class', 'Text']]

In [ ]:
train_df

In [ ]:
test_df

In [ ]:

def clean_text(text):    
    text = re.sub(r"\n+", " ", text)
    text = re.sub(r"\t+", " ", text)
    text = re.sub(r"\r+", " ", text)
    text = re.sub(r"/\sThe+", "The", text)
    #text = re.sub(r"(\\')+", "'", text)
    text = re.sub(r'\\+', '', text)
    
    return text

In [ ]:
yelp = pd.read_csv('./data/original/yelp/yelp.csv')
yelp = yelp[['stars', 'text']]
yelp['text'] = yelp['text'].map(clean_text)
yelp.columns = ['class', 'text']
yelp['text'] = yelp['text'].apply(lambda x: str(x).replace("\\'s","'s"))

In [ ]:
yelp

In [ ]:
train, test = train_test_split(yelp, test_size=0.1, random_state=42)
train.to_csv('./data/original/bbc/train.csv', index=False)
test.to_csv('./data/original/bbc/test.csv', index=False)

np.savetxt('./data/original/bbc/train.txt', train.values, fmt='%s', delimiter='\t')
np.savetxt('./data/original/bbc/test.txt', test.values, fmt='%s', delimiter='\t')

In [ ]:
for i in range (10):
    print(yelp['class'].iloc[i])

In [ ]:
train['text'].iloc[2248]

In [ ]:
txt = 'There are a few nice semi-upscale restaurants here. It\'s also got a really good swanky (and somewhat hidden) bar here.'
new_txt  = clean_text(txt)

In [ ]:
new_txt

In [ ]:
# augment
aug_samples('yelp', 0.0001)

In [ ]:
def aug_samples(dataset_name, method_names, fraction, pct_words_to_swap, transformations_per_example, include_original):
    data = load_data(dict_datasets[dataset_name])
    print(f'Loaded {len(data)} rows of data from {dict_datasets[dataset_name]}')
    #data = data.sample(frac=fraction)
    #print(f'Sampled {len(data)} rows with fraction={fraction}')
    for method in method_names:
        if method not in dict_methods:
            raise ValueError(f"Method '{method}' is not supported")
        augmented_data = augment_text(
            data,
            dict_methods[method],
            fraction=fraction,
            pct_words_to_swap=pct_words_to_swap,
            transformations_per_example=transformations_per_example,
            label_column='class',
            target_column='text',
            include_original=include_original
        )
        augmented_data = augmented_data[['class', 'text']]

        print(f'number of augmented samples: {len(augmented_data)}')

In [ ]:
import numpy as np
import pandas as pd
import textattack
import logging
from functions import *
def test_augmentation_methods():
    # Define the input paths for the datasets
    dict_datasets = {
        'agnews': './data/original/agnews/train.txt',
        'cardio': './data/original/cardio/train.txt',
        'cr': './data/original/cr/train.txt',
        'pc': './data/original/pc/train.txt',
        'pc_clean': './data/original/pc/train_clean.txt',
        'sst2': './data/original/sst2/train.txt',
        'subj': './data/original/subj/train.txt',
        'trec': './data/original/trec/train.txt',
        'kaggle_med': './data/original/kaggle_med/train.txt',
        'yelp': './data/original/yelp/train.txt',
        'bbc': './data/original/bbc/train.txt'
    }

    # Define the augmentation methods and their names
    dict_methods = {
        'aeda': 'aeda_augmenter',
        'backtranslation': 'backtranslation_augmenter',
        'checklist': 'checklist_augmenter',
        'clare': 'clare_augmenter',
        'eda': 'eda_augmenter',
        'embedding': 'embedding_augmenter',
        'deletion': 'deletion_augmenter',
        'wordnet': 'wordnet_augmenter'
    }

    datasets_with_samples = []
    methods_with_samples = []

    for dataset_name in dict_datasets:
        path = dict_datasets[dataset_name]
        data = pd.read_csv(path, delimiter='\t', header=None, names=['class', 'text'])
        print(f'Loaded {len(data)} rows of data from {dict_datasets[dataset_name]}')

        # Take only one sample from each dataset
        data = data.sample(n=1)

        for method in dict_methods:
            augmented_data = augment_text(
                data,
                dict_methods[method],
                fraction=1,
                pct_words_to_swap=0.1,
                transformations_per_example=1,
                label_column='class',
                target_column='text',
                include_original=False
            )
            augmented_data = augmented_data[['class', 'text']]
            
            if len(augmented_data) > 0:
                print(f"Generated {len(augmented_data)} samples for dataset '{dataset_name}' and method '{method}'")
                datasets_with_samples.append(dataset_name)
                methods_with_samples.append(method)
    
    datasets_with_samples = set(datasets_with_samples)
    methods_with_samples = set(methods_with_samples)
    
    print(f"\nDatasets with samples: {', '.join(datasets_with_samples)}")
    print(f"Methods with samples: {', '.join(methods_with_samples)}")
    print(f"\nDatasets without any samples: {', '.join(set(dict_datasets.keys()) - datasets_with_samples)}")
    print(f"Methods without any samples: {', '.join(set(dict_methods.keys()) - methods_with_samples)}")


In [ ]:
import numpy as np
import pandas as pd
import textattack
import logging
from functions import *
def test_augmentation_methods():
    # Define the input paths for the datasets
    dict_datasets = {
        'agnews': './data/original/agnews/train.txt',
        'cardio': './data/original/cardio/train.txt',
        'cr': './data/original/cr/train.txt',
        'pc': './data/original/pc/train.txt',
        'pc_clean': './data/original/pc/train_clean.txt',
        'sst2': './data/original/sst2/train.txt',
        'subj': './data/original/subj/train.txt',
        'trec': './data/original/trec/train.txt',
        'kaggle_med': './data/original/kaggle_med/train.txt',
        'yelp': './data/original/yelp/train.txt',
        'bbc': './data/original/bbc/train.txt'
    }

    # Define the augmentation methods and their names
    dict_methods = {
        'aeda': 'aeda_augmenter',
        'backtranslation': 'backtranslation_augmenter',
        'checklist': 'checklist_augmenter',
        'clare': 'clare_augmenter',
        'eda': 'eda_augmenter',
        'wordnet': 'wordnet_augmenter'
    }

    datasets_with_samples = []
    methods_with_samples = []
    results = []

    for dataset_name in dict_datasets:
        path = dict_datasets[dataset_name]
        data = pd.read_csv(path, delimiter='\t', header=None, names=['class', 'text'])
        print(f'Loaded {len(data)} rows of data from {dict_datasets[dataset_name]}')

        # Take only one sample from each dataset
        data = data.sample(n=1)
        print(data)

        for method in dict_methods:
            print('=====================================')
            print(f"Augmenting dataset '{dataset_name}' with method '{method}'")
            print('=====================================')
            augmented_data = augment_text(
                data,
                dict_methods[method],
                fraction=1,
                pct_words_to_swap=0.1,
                transformations_per_example=1,
                label_column='class',
                target_column='text',
                include_original=False
            )
            augmented_data = augmented_data[['class', 'text']]
            
            if len(augmented_data) > 0:
                print(f"Generated {len(augmented_data)} samples for dataset '{dataset_name}' and method '{method}'")
                datasets_with_samples.append(dataset_name)
                methods_with_samples.append(method)
                result = {
                    'dataset_name': dataset_name,
                    'method_name': method,
                    'num_samples': len(augmented_data)
                }
                results.append(result)
    
    datasets_with_samples = set(datasets_with_samples)
    methods_with_samples = set(methods_with_samples)
    
    print(f"\nDatasets with samples: {', '.join(datasets_with_samples)}")
    print(f"Methods with samples: {', '.join(methods_with_samples)}")
    print(f"\nDatasets without any samples: {', '.join(set(dict_datasets.keys()) - datasets_with_samples)}")
    print(f"Methods without any samples: {', '.join(set(dict_methods.keys()) - methods_with_samples)}")
    
    # Create a dataframe with the results
    df_results = pd.DataFrame(results)
    
    return df_results

import sys

# Redirect stderr to /dev/null
sys.stderr = open('/dev/null', 'w')


In [ ]:
results = test_augmentation_methods()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

def clare_augmentation(text, pct_words_to_swap, transformations_per_example):
    # Load pre-trained masked language model
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

    # Tokenize text
    tokens = tokenizer.tokenize(text)

    # Replace a fraction of tokens with [MASK] tokens
    num_words_to_swap = int(pct_words_to_swap * len(tokens))
    indices_to_swap = set()
    while len(indices_to_swap) < num_words_to_swap:
        index = torch.randint(len(tokens), size=(1,)).item()
        if index not in indices_to_swap:
            indices_to_swap.add(index)
            tokens[index] = "[MASK]"

    # Generate transformed text examples
    transformed_texts = set()
    for i in range(transformations_per_example):
        # Mask some tokens
        masked_tokens = tokens.copy()
        for index in indices_to_swap:
            masked_tokens[index] = "[MASK]"

        # Convert tokens to ids
        input_ids = tokenizer.convert_tokens_to_ids(masked_tokens)
        input_ids = torch.tensor([input_ids])

        # Get predicted token probabilities from model
        with torch.no_grad():
            outputs = model(input_ids)
            predictions = outputs[0][0]

        # Generate new tokens by sampling from predicted probabilities
        new_tokens = []
        for index in indices_to_swap:
            probabilities = predictions[index]
            probabilities /= probabilities.sum()
            predicted_token_id = torch.multinomial(probabilities, 1).item()
            predicted_token = tokenizer.convert_ids_to_tokens([predicted_token_id])[0]
            new_tokens.append(predicted_token)
        for index, token in zip(indices_to_swap, new_tokens):
            masked_tokens[index] = token

        # Convert tokens back to text
        transformed_text = tokenizer.convert_tokens_to_string(masked_tokens)
        transformed_texts.add(transformed_text)

    return list(transformed_texts)


In [ ]:
data = pd.read_csv('./data/original/agnews/train.txt', delimiter='\t', header=None, names=['class', 'text'])
data = data.sample(n=1)['text'].values[0]
print(data)


In [ ]:
txt

In [ ]:
res = clare_augmentation(data, 0.1, 1)

In [ ]:
print (txt)

In [ ]:
import torch
import transformers

def clare_augmentation(text, pct_words_to_swap, transformations_per_example):
    # Load pre-trained BERT model and tokenizer
    model_name = 'bert-base-uncased'
    tokenizer = transformers.BertTokenizer.from_pretrained(model_name)
    model = transformers.BertForMaskedLM.from_pretrained(model_name)
    model.eval()

    # Preprocess text
    text = text.replace('#', '')  # Remove special characters
    text = text.strip()  # Remove leading/trailing whitespace

    # Tokenize input text
    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    # Select indices of words to perturb
    num_words = len(token_ids)
    indices_to_modify = set()
    while len(indices_to_modify) < int(num_words * pct_words_to_swap):
        indices_to_modify.add(torch.randint(num_words, size=(1,)).item())

    # Perturb words
    new_texts = []
    for _ in range(transformations_per_example):
        new_tokens = []
        for i, token_id in enumerate(token_ids):
            if i in indices_to_modify:
                # Replace token with predicted token from BERT model
                predictions = model(torch.tensor([token_ids])).logits[0]
                probabilities = predictions[i]
                probabilities /= probabilities.sum()
                predicted_token_id = torch.multinomial(probabilities, 1).item()
                predicted_token = tokenizer.convert_ids_to_tokens([predicted_token_id])[0]
                new_tokens.append(predicted_token)
            else:
                new_tokens.append(tokens[i])

        new_text = tokenizer.convert_tokens_to_string(new_tokens)
        new_texts.append(new_text)

    return new_texts


In [ ]:
text = "Not too many 27-year-old rookies come down the pike \
    in the NBA. But Nets coach Lawrence Frank wasn #39;t concerned \
      that Awvee Storey #39;s only previous NBA experience was a summer camp with the Dallas Mavericks."
augmented_texts = clare_augmentation(text, pct_words_to_swap=0.5, transformations_per_example=1)
print(augmented_texts)


In [ ]:
text = "Not too many 27-year-old rookies come down the pike in the NBA. But Nets coach Lawrence Frank wasn #39;t concerned that Awvee Storey #39;s only previous NBA experience was a summer camp with the Dallas Mavericks."
augmented_texts = clare_augmentation(text, pct_words_to_swap=0.5, transformations_per_example=1)
print(augmented_texts)


In [ ]:
text = "Not too many 27-year-old rookies come down the pike in the NBA. But Nets coach Lawrence Frank wasn #39;t concerned that Awvee Storey #39;s only previous NBA experience was a summer camp with the Dallas Mavericks."
augmented_texts = clare_augmentation(text, pct_words_to_swap=0.5, transformations_per_example=1)
print(augmented_texts)


In [ ]:
text = "The quick brown fox jumps over the lazy dog."
augmented_texts = clare_augmentation(text, pct_words_to_swap=0.5, transformations_per_example=1)
print(augmented_texts)


In [ ]:

text = "The quick brown fox jumps over the lazy dog."
augmented_texts = clare_augmentation(text,model_name='bert-large-cased', pct_words_to_swap=0.5, transformations_per_example=1)

print(augmented_texts)


In [ ]:
import numpy as np
import pandas as pd
from functions import *

dict_datasets = {
    'agnews': './data/original/agnews/train.txt',
    #'cardio': './data/original/cardio/train.txt',
    'cr': './data/original/cr/train.txt',
    'pc': './data/original/pc/train.txt',
    'pc_clean': './data/original/pc/train_clean.txt',
    'sst2': './data/original/sst2/train.txt',
    'subj': './data/original/subj/train.txt',
    'trec': './data/original/trec/train.txt',
    'kaggle_med': './data/original/kaggle_med/train.txt',
    'yelp': './data/original/yelp/train.txt',
    'bbc': './data/original/bbc/train.txt'
}

# Define the augmentation methods and their names
dict_methods = {
    'aeda': 'aeda_augmenter',
    'backtranslation': 'backtranslation_augmenter',
    'checklist': 'checklist_augmenter',
    'clare': 'clare_augmenter',
    'eda': 'eda_augmenter',
    #'embedding': 'embedding_augmenter',
    #'deletion': 'deletion_augmenter',
    'wordnet': 'wordnet_augmenter'
}

In [ ]:
df = load_data(dict_datasets['yelp'])
df = df.sample(frac=0.01, random_state=42)
len(df)

In [ ]:
import clare
aug = clare.Clare_Augmenter(pct_words_to_swap=0.5, transformations_per_example=1)
for text in df['text']:
    print(aug.augment(text))
# res = aug.augment(df)

In [ ]:
import clare
aug = clare.Clare_Augmenter(pct_words_to_swap=0.5, transformations_per_example=5)
for text in df['text']:
    print(aug.augment(text))
# res = aug.augment(df)

In [ ]:
import numpy as np
import pandas as pd
from functions import *
df = load_data('./data/original/agnews/train.txt')
df = df.sample(frac=0.0001, random_state=42)


In [ ]:
aug_df = augment_text(df,'clare_augmenter',fraction=1, pct_words_to_swap=0.5, transformations_per_example=1)

In [ ]:
from clare import Clare_Augmenter
df = load_data('./data/original/agnews/train.txt')
df = df.sample(frac=0.0001, random_state=42)
sentence = df['text'].iloc[0]
aug = Clare_Augmenter(pct_words_to_swap=0.5, transformations_per_example=1)
aug_df = aug.augment(sentence)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import BertForSequenceClassification, BertTokenizer
class Clare_Augmenter:
    def __init__(self, num_labels=2, pct_words_to_swap=0.1, transformations_per_example=1):
        self.pct_words_to_swap = pct_words_to_swap
        self.transformations_per_example = transformations_per_example
        self.num_labels = num_labels

    def augment(self,text,model_name='bert-base-uncased'):
     
        tokenizer = BertTokenizer.from_pretrained(model_name)
        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=self.num_labels)
        
        # Tokenize the input text
        tokens = tokenizer.tokenize(text)
        
        new_texts = []
        for _ in range(self.transformations_per_example):
            new_tokens = []
            for i, token in enumerate(tokens):
                # Decide whether to perturb this token
                if torch.rand(1).item() < self.pct_words_to_swap:
                    # Replace the token with a random token from the BERT vocabulary
                    masked_tokens = ["[MASK]"] * len(tokens)
                    masked_tokens[i] = token
                    masked_input = " ".join(masked_tokens)
                    input_ids = tokenizer.encode(masked_input, return_tensors="pt")
                    with torch.no_grad():
                        predictions = model(input_ids)[0][0, i].reshape(-1)  # reshape predictions
                    probabilities = predictions.softmax(dim=0)
                    predicted_token_id = torch.multinomial(probabilities, 1).item()
                    predicted_token = tokenizer.convert_ids_to_tokens([predicted_token_id])[0]
                    new_tokens.append(predicted_token)
                else:
                    new_tokens.append(token)
            
            # Convert the new tokens back to text and add it to the list of augmented texts
            new_text = tokenizer.convert_tokens_to_string(new_tokens)
            new_texts.append(new_text)
        
        return new_texts


In [ ]:
df = load_data('./data/original/agnews/train.txt')
df = df.sample(frac=0.0001, random_state=42)
sentence = df['text'].iloc[0]
aug = Clare_Augmenter(pct_words_to_swap=0.5, transformations_per_example=1)
aug_df = aug.augment(sentence)

In [ ]:
class Clare_Augmenter:
    def __init__(self, num_labels=2, pct_words_to_swap=0.1, transformations_per_example=1):
        self.pct_words_to_swap = pct_words_to_swap
        self.transformations_per_example = transformations_per_example
        self.num_labels = num_labels

    def augment(self,text,model_name='bert-base-uncased'):
     
        tokenizer = BertTokenizer.from_pretrained(model_name, max_length=1024)
        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=self.num_labels)
        
        # Tokenize the input text
        tokens = tokenizer.tokenize(text)
        
        new_texts = []
        for _ in range(self.transformations_per_example):
            new_tokens = []
            for i, token in enumerate(tokens):
                # Decide whether to perturb this token
                if torch.rand(1).item() < self.pct_words_to_swap:
                    # Replace the token with a random token from the BERT vocabulary
                    masked_tokens = ["[MASK]"] * len(tokens)
                    masked_tokens[i] = token
                    masked_input = " ".join(masked_tokens)
                    input_ids = tokenizer.encode(masked_input, return_tensors="pt")
                    with torch.no_grad():
                        predictions = model(input_ids)[0][0, i].reshape(-1)  # reshape predictions
                    probabilities = predictions.softmax(dim=0)
                    predicted_token_id = torch.multinomial(probabilities, 1).item()
                    predicted_token = tokenizer.convert_ids_to_tokens([predicted_token_id])[0]
                    new_tokens.append(predicted_token)
                else:
                    new_tokens.append(token)
            
            # Convert the new tokens back to text and add it to the list of augmented texts
            new_text = tokenizer.convert_tokens_to_string(new_tokens)
            new_texts.append(new_text)
        
        return new_texts


In [ ]:
df = load_data('./data/original/agnews/train.txt')
df = df.sample(frac=0.0001, random_state=42)
sentence = df['text'].iloc[0]
aug = Clare_Augmenter(pct_words_to_swap=0.5, transformations_per_example=1)
aug_df = aug.augment(sentence)

In [ ]:
class Clare_Augmenter:
    def __init__(self, pct_words_to_swap=0.1, transformations_per_example=1):
        self.pct_words_to_swap = pct_words_to_swap
        self.transformations_per_example = transformations_per_example
        
        # Initialize the tokenizer and model
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        self.model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
        self.model.eval()

    def augment(self, text):
        # Tokenize the input text
        tokens = self.tokenizer.tokenize(text)
        
        new_texts = []
        for _ in range(self.transformations_per_example):
            new_tokens = []
            for i, token in enumerate(tokens):
                # Decide whether to perturb this token
                if torch.rand(1).item() < self.pct_words_to_swap:
                    # Replace the token with a random token from the BERT vocabulary
                    masked_tokens = ["[MASK]"] * len(tokens)
                    masked_tokens[i] = token
                    masked_input = " ".join(masked_tokens)
                    input_ids = self.tokenizer.encode(masked_input, return_tensors="pt")
                    with torch.no_grad():
                        predictions = self.model(input_ids).logits[0, i].reshape(-1)  # reshape predictions
                    probabilities = predictions.softmax(dim=0)
                    predicted_token_id = torch.multinomial(probabilities, 1).item()
                    predicted_token = self.tokenizer.convert_ids_to_tokens([predicted_token_id])[0]
                    new_tokens.append(predicted_token)
                else:
                    new_tokens.append(token)
            
            # Convert the new tokens back to text and add it to the list of augmented texts
            new_text = self.tokenizer.convert_tokens_to_string(new_tokens)
            new_texts.append(new_text)
        
        return new_texts


In [ ]:
df = load_data('./data/original/agnews/train.txt')
df = df.sample(frac=0.0001, random_state=42)
sentence = df['text'].iloc[0]
aug = Clare_Augmenter(pct_words_to_swap=0.5, transformations_per_example=1)
aug_df = aug.augment(sentence)

In [ ]:
class Clare_Augmenter:
    def __init__(self, pct_words_to_swap=0.1, transformations_per_example=1):
        self.pct_words_to_swap = pct_words_to_swap
        self.transformations_per_example = transformations_per_example
        
        # Initialize the tokenizer and model
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", max_length=1024)
        self.model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
        self.model.eval()

    def augment(self, text):
        # Tokenize the input text
        tokens = self.tokenizer.tokenize(text)
        
        new_texts = []
        for _ in range(self.transformations_per_example):
            new_tokens = []
            for i, token in enumerate(tokens):
                # Decide whether to perturb this token
                if torch.rand(1).item() < self.pct_words_to_swap:
                    # Replace the token with a random token from the BERT vocabulary
                    masked_tokens = ["[MASK]"] * len(tokens)
                    masked_tokens[i] = token
                    masked_input = " ".join(masked_tokens)
                    input_ids = self.tokenizer.encode(masked_input, return_tensors="pt")
                    with torch.no_grad():
                        predictions = self.model(input_ids).logits[0, i].reshape(-1)  # reshape predictions
                    probabilities = predictions.softmax(dim=0)
                    predicted_token_id = torch.multinomial(probabilities, 1).item()
                    predicted_token = self.tokenizer.convert_ids_to_tokens([predicted_token_id])[0]
                    new_tokens.append(predicted_token)
                else:
                    new_tokens.append(token)
            
            # Convert the new tokens back to text and add it to the list of augmented texts
            new_text = self.tokenizer.convert_tokens_to_string(new_tokens)
            new_texts.append(new_text)
        
        return new_texts


In [ ]:
df = load_data('./data/original/agnews/train.txt')
df = df.sample(frac=0.0001, random_state=42)
sentence = df['text'].iloc[0]
aug = Clare_Augmenter(pct_words_to_swap=0.5, transformations_per_example=1)
aug_df = aug.augment(sentence)

In [ ]:
def clare_augmentation(text, pct_words_to_swap, transformations_per_example, max_length=512):
    # Load pre-trained BERT model and tokenizer
    model_name = 'bert-base-uncased'
    tokenizer = transformers.BertTokenizer.from_pretrained(model_name)
    model = transformers.BertForMaskedLM.from_pretrained(model_name)
    model.eval()

    # Preprocess text
    text = text.replace('#', '')  # Remove special characters
    text = text.strip()  # Remove leading/trailing whitespace

    # Tokenize input text
    tokens = tokenizer(text, truncation=True, max_length=max_length, return_tensors='pt').input_ids[0].tolist()
    tokens = tokenizer.convert_ids_to_tokens(tokens)

    # Select indices of words to perturb
    num_words = len(tokens)
    indices_to_modify = set()
    while len(indices_to_modify) < int(num_words * pct_words_to_swap):
        indices_to_modify.add(torch.randint(num_words, size=(1,)).item())

    # Perturb words
    new_texts = []
    for _ in range(transformations_per_example):
        new_tokens = []
        for i, token_id in enumerate(tokens):
            if i in indices_to_modify:
                # Replace token with predicted token from BERT model
                predictions = model(torch.tensor([tokens])).logits[0]
                probabilities = predictions[i]
                probabilities /= probabilities.sum()
                predicted_token_id = torch.multinomial(probabilities, 1).item()
                predicted_token = tokenizer.convert_ids_to_tokens([predicted_token_id])[0]
                new_tokens.append(predicted_token)
            else:
                new_tokens.append(tokenizer.convert_ids_to_tokens([token_id], skip_special_tokens=True)[0])


        new_text = tokenizer.convert_tokens_to_string(new_tokens)
        new_texts.append(new_text)

    return new_texts


In [ ]:
txt = 'The quick brown fox jumps over the lazy dog.'
clare_augmentation(txt, pct_words_to_swap=0.5, transformations_per_example=1)

In [ ]:
import torch
import transformers


def clare_augmentation(text, pct_words_to_swap, transformations_per_example):
    # Load pre-trained BERT model and tokenizer
    model_name = 'bert-base-uncased'
    tokenizer = transformers.BertTokenizer.from_pretrained(model_name)
    model = transformers.BertForMaskedLM.from_pretrained(model_name)
    model.eval()

    # Preprocess text
    text = text.replace('#', '')  # Remove special characters
    text = text.strip()  # Remove leading/trailing whitespace

    # Tokenize input text
    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    # Select indices of words to perturb
    num_words = len(token_ids)
    indices_to_modify = set()
    while len(indices_to_modify) < int(num_words * pct_words_to_swap):
        indices_to_modify.add(torch.randint(num_words, size=(1,)).item())

    # Perturb words
    new_texts = []
    for _ in range(transformations_per_example):
        new_tokens = []
        for i, token_id in enumerate(token_ids):
            if i in indices_to_modify:
                # Replace token with predicted token from BERT model
                with torch.no_grad():
                    predictions = model(torch.tensor([token_ids])).logits[0]
                    probabilities = predictions[i].softmax(dim=0)
                    predicted_token_id = torch.multinomial(probabilities, 1).item()
                    predicted_token = tokenizer.convert_ids_to_tokens([predicted_token_id])[0]
                    new_tokens.append(predicted_token)
            else:
                new_tokens.append(tokenizer.convert_ids_to_tokens([token_id])[0])

        new_text = tokenizer.convert_tokens_to_string(new_tokens)
        new_texts.append(new_text)

    return new_texts


In [ ]:
txt = 'The quick brown fox jumps over the lazy dog.'
clare_augmentation(txt, pct_words_to_swap=0.5, transformations_per_example=1)

In [ ]:
import numpy as np
import pandas as pd
from functions import *
df = load_data('./data/original/agnews/train.txt')
df = df.sample(frac=0.0001, random_state=42)
sentence = df['text'].iloc[0]
aug = Clare_Augmenter(pct_words_to_swap=0.5, transformations_per_example=1)
aug_df = aug.augment(sentence)

In [ ]:
aug_df

In [ ]:
import numpy as np
import pandas as pd
import textattack
import logging
from functions import *
def test_augmentation_methods():
    # Define the input paths for the datasets
    dict_datasets = {
        'agnews': './data/original/agnews/train.txt',
        # 'cardio': './data/original/cardio/train.txt',
        # 'cr': './data/original/cr/train.txt',
        # 'pc': './data/original/pc/train.txt',
        # 'pc_clean': './data/original/pc/train_clean.txt',
        # 'sst2': './data/original/sst2/train.txt',
        # 'subj': './data/original/subj/train.txt',
        # 'trec': './data/original/trec/train.txt',
        # 'kaggle_med': './data/original/kaggle_med/train.txt',
        # 'yelp': './data/original/yelp/train.txt',
        # 'bbc': './data/original/bbc/train.txt'
    }

    # Define the augmentation methods and their names
    dict_methods = {
        # 'aeda': 'aeda_augmenter',
        # 'backtranslation': 'backtranslation_augmenter',
        # 'checklist': 'checklist_augmenter',
         'clare': 'clare_augmenter',
        # 'eda': 'eda_augmenter',
        # 'wordnet': 'wordnet_augmenter'
    }

    datasets_with_samples = []
    methods_with_samples = []
    results = []

    for dataset_name in dict_datasets:
        path = dict_datasets[dataset_name]
        data = pd.read_csv(path, delimiter='\t', header=None, names=['class', 'text'])
        print(f'Loaded {len(data)} rows of data from {dict_datasets[dataset_name]}')

        # Take only one sample from each dataset
        data = data.sample(n=1)
        print(data)

        for method in dict_methods:
            print('=====================================')
            print(f"Augmenting dataset '{dataset_name}' with method '{method}'")
            print('=====================================')
            augmented_data = augment_text(
                data,
                dict_methods[method],
                fraction=1,
                pct_words_to_swap=0.1,
                transformations_per_example=1,
                label_column='class',
                target_column='text',
                include_original=False
            )
            augmented_data = augmented_data[['class', 'text']]
            
            if len(augmented_data) > 0:
                print(f"Generated {len(augmented_data)} samples for dataset '{dataset_name}' and method '{method}'")
                datasets_with_samples.append(dataset_name)
                methods_with_samples.append(method)
                result = {
                    'dataset_name': dataset_name,
                    'method_name': method,
                    'num_samples': len(augmented_data)
                }
                results.append(result)
    
    datasets_with_samples = set(datasets_with_samples)
    methods_with_samples = set(methods_with_samples)
    
    print(f"\nDatasets with samples: {', '.join(datasets_with_samples)}")
    print(f"Methods with samples: {', '.join(methods_with_samples)}")
    print(f"\nDatasets without any samples: {', '.join(set(dict_datasets.keys()) - datasets_with_samples)}")
    print(f"Methods without any samples: {', '.join(set(dict_methods.keys()) - methods_with_samples)}")
    
    # Create a dataframe with the results
    df_results = pd.DataFrame(results)
    
    return df_results

import sys

# Redirect stderr to /dev/null
sys.stderr = open('/dev/null', 'w')


In [ ]:
test_augmentation_methods()


In [ ]:
import numpy as np
import pandas as pd
import textattack
import logging
from functions import *

In [ ]:
dict_datasets = {
    'agnews': './data/original/agnews/train.txt',
    'cardio': './data/original/cardio/train.txt',
    'cr': './data/original/cr/train.txt',
    'pc': './data/original/pc/train.txt',
    #'pc_clean': './data/original/pc/train_clean.txt',
    'sst2': './data/original/sst2/train.txt',
    'subj': './data/original/subj/train.txt',
    'trec': './data/original/trec/train.txt',
    'kaggle_med': './data/original/kaggle_med/train.txt',
    'yelp': './data/original/yelp/train.txt',
    'bbc': './data/original/bbc/train.txt'
}

# Define the augmentation methods and their names
dict_methods = {
    'aeda': 'aeda_augmenter',
    'backtranslation': 'backtranslation_augmenter',
    'checklist': 'checklist_augmenter',
    'clare': 'clare_augmenter',
    'eda': 'eda_augmenter',
    'wordnet': 'wordnet_augmenter',
    'charswap': 'charswap_augmenter',
    'deletion': 'deletion_augmenter',
    'embedding': 'embedding_augmenter'
}

In [ ]:
df = load_data('./data/original/agnews/train.txt')
df = df.sample(n=1, random_state=42)
method = dict_methods['backtranslation']

In [ ]:
len(df)

In [ ]:
df

In [ ]:
df

In [ ]:
import transformers
transformers.logging.set_verbosity_error()
import warnings
warnings.filterwarnings('ignore')

res = augment_text(df, method, fraction=1, pct_words_to_swap=0.5, transformations_per_example=1, label_column='class', target_column='text', include_original=True)


In [ ]:
res

In [ ]:
res

In [ ]:
for i in dict_methods:
    for j in dict_datasets:
        df = load_data(dict_datasets[j])
        df = df.sample(n=1)        
        method = dict_methods[i]
        print(f'augmenting {j} with {i} \n ')
        res = augment_text(df, method, fraction=1, pct_words_to_swap=0.5, transformations_per_example=1, label_column='class', target_column='text', include_original=True)
        print('=====================================')
        print(f'results for {j} with {i}: \n {res}')
        print('=====================================')

## convert all txt files to csv

In [ ]:
dict_datasets = {
    'agnews': './data/original/agnews/test.txt',
    'cardio': './data/original/cardio/test.txt',
    'cr': './data/original/cr/test.txt',
    'pc': './data/original/pc/test.txt',
    #'pc_clean': './data/original/pc/test_clean.txt',
    'sst2': './data/original/sst2/test.txt',
    'subj': './data/original/subj/test.txt',
    'trec': './data/original/trec/test.txt',
    'kaggle_med': './data/original/kaggle_med/test.txt',
    'yelp': './data/original/yelp/test.txt',
    'bbc': './data/original/bbc/test.txt'
}


In [ ]:
# convert all datasets to csv if it is not already existing
for i in dict_datasets:
    path = dict_datasets[i]
    if os.path.exists(f'./data/original/{i}/test.csv'):
        print(f'./data/original/{i}/test.csv already exists')
    else:
        print(f'./data/original/{i}/test.csv does not exist')
        print(f'creating ./data/original/{i}/test.csv')
        df = load_data(path)
        df.to_csv(f'./data/original/{i}/test.csv', index=False)
        print('done')



In [ ]:
def txt_to_csv(txt_file):
    '''
    Return a csv file from a txt file with the same name
    with class and text separated by a comma.
    '''
    if txt_file.endswith('.txt'):
        try:
            df = pd.read_csv(txt_file, sep='\t', header=None, names=['class', 'text'])
        except pd.errors.ParserError:
            df = pd.read_csv(txt_file, sep=' ', header=None, names=['class', 'text'], engine='python')
        df.to_csv(txt_file[:-4] + '.csv', index=False)
    else:
        raise ValueError('File format not supported.')

In [ ]:
txt_file = './data/original/cr/test.txt'
df = pd.read_csv(txt_file, sep='\t', header=None, names=['class', 'text'])



In [ ]:
load_data('./data/original/cr/test.txt')

## check all datasets with methods with csv format

In [1]:
import numpy as np
import pandas as pd
import textattack
import logging
from functions import *

In [2]:
dict_datasets = {
    'agnews': './data/original/agnews/train.csv',
    'cardio': './data/original/cardio/train.csv',
    'cr': './data/original/cr/train.csv',
    'pc': './data/original/pc/train.csv',
    'sst2': './data/original/sst2/train.csv',
    'subj': './data/original/subj/train.csv',
    'trec': './data/original/trec/train.csv',
    'kaggle_med': './data/original/kaggle_med/train.csv',
    'yelp': './data/original/yelp/train.csv',
    'bbc': './data/original/bbc/train.csv'
}
dict_methods = {
    'aeda': 'aeda_augmenter',
    'backtranslation': 'backtranslation_augmenter',
    'checklist': 'checklist_augmenter',
    'clare': 'clare_augmenter',
    'eda': 'eda_augmenter',
    'wordnet': 'wordnet_augmenter',
    'charswap': 'charswap_augmenter',
    'deletion': 'deletion_augmenter',
    'embedding': 'embedding_augmenter'
}

In [4]:
for i in dict_methods:
    for j in dict_datasets:        
        df = load_data(dict_datasets[j])
        df = df.sample(n=1)        
        method = dict_methods[i]
        print(f'augmenting {j} with {i} \n ')
        res = augment_text(df, method, fraction=1, pct_words_to_swap=0.5, transformations_per_example=1, label_column='class', target_column='text', include_original=True)
        print('=====================================')
        print(f'results for {j} with {i}: \n {res}')
        print('=====================================')

augmenting agnews with aeda 
 


[nltk_data] Downloading package omw-1.4 to /home/peyman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/peyman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/home/peyman/anaconda3/envs/test2/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


results for agnews with aeda: 
                                                 text  class
0  AFP - Women activists urged Sri Lankan authori...      1
1  . AFP ? - Women : activists urged ! Sri , Lank...      1
augmenting cardio with aeda 
 


[nltk_data] Downloading package omw-1.4 to /home/peyman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/peyman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/home/peyman/anaconda3/envs/test2/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


results for cardio with aeda: 
                                                 text  class
0   This study compares the metabolism and functi...     14
1   This . study compares : the metabolism ! and ...     14
augmenting cr with aeda 
 


...

results for kaggle_med with embedding: 
                                          text  class
0      Colonoscopy to screen for colon cancer     14
1   Colonoscopy to screen for surfboard tumor     14
augmenting yelp with embedding 
 


[nltk_data] Downloading package omw-1.4 to /home/peyman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/peyman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/home/peyman/anaconda3/envs/test2/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


results for yelp with embedding: 
                                                 text  class
0  I was also roped in with a flier on my door, a...      5
1  I was similarly roped in with a prospectus on ...      5
augmenting bbc with embedding 
 


[nltk_data] Downloading package omw-1.4 to /home/peyman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/peyman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/home/peyman/anaconda3/envs/test2/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


results for bbc with embedding: 
                                                 text  class
0  There is no shortage of Mexican restaurants in...      5
1  There is no imperfection of Wetback restaurant...      5


In [ ]:
for i in dict_methods:
    for j in dict_datasets:
        df = load_data(dict_datasets[j])
        if df is None:
            continue
        df = df.sample(n=1)
        method = dict_methods[i]
        print(f'augmenting {j} with {i} \n ')
        try:
            res = augment_text(df, method, fraction=1, pct_words_to_swap=0.5, transformations_per_example=1, label_column='class', target_column='text', include_original=True)
            print('=====================================')
            print(f'results for {j} with {i}: \n {res}')
            print('=====================================')
        except IndexError as e:
            print(f"IndexError in dataset {j} with method {i}: {e}")
            continue


In [3]:
df = load_data('./data/original/cardio/train.csv')

In [ ]:
df.to_csv('./data/original/cardio/train.csv', index=False)

In [ ]:
df = pd.read_csv('./data/original/cardio/train.csv', header=None, names=['class','text'], dtype={'class': str},skiprows=1)

In [ ]:
df

## prepare the data fractions

In [9]:
dict_datasets = {
    'agnews': './data/original/agnews/train.csv',
    'cardio': './data/original/cardio/train.csv',
    'cr': './data/original/cr/train.csv',
    'pc': './data/original/pc/train.csv',    
    'sst2': './data/original/sst2/train.csv',
    'subj': './data/original/subj/train.csv',
    'trec': './data/original/trec/train.csv',
    'kaggle_med': './data/original/kaggle_med/train.csv',
    'yelp': './data/original/yelp/train.csv',
    'bbc': './data/original/bbc/train.csv'
}    

In [11]:
random_state = 20
for i in dict_datasets:
    df = load_data(dict_datasets[i])
    df = df.sample(frac=0.1, random_state=random_state)
    df.to_csv(f'./data/original/{i}/train_10.csv', index=False)
    df = df.sample(frac=0.2, random_state=random_state)
    df.to_csv(f'./data/original/{i}/train_20.csv', index=False)
    df = df.sample(frac=0.5, random_state=random_state)
    df.to_csv(f'./data/original/{i}/train_50.csv', index=False)


## sort the datasets by the number of samples.

In [15]:
new_dict_datasets = {}

for i in dict_datasets:
    len_dataset = len(load_data(dict_datasets[i]))
    new_dict_datasets[i] = len_dataset

new_dict_datasets = sorted(new_dict_datasets.items(), key=lambda x: x[1])



In [16]:
new_dict_datasets

[('cr', 4067),
 ('kaggle_med', 4245),
 ('trec', 5452),
 ('sst2', 7791),
 ('subj', 9000),
 ('yelp', 9000),
 ('bbc', 9000),
 ('pc', 39879),
 ('agnews', 120000),
 ('cardio', 435002)]

In [ ]:
dict_datasets = {
    'agnews': './data/original/agnews/train.txt',
    'cardio': './data/original/cardio/train.txt',
    'cr': './data/original/cr/train.txt',
    'pc': './data/original/pc/train.txt',
    'pc_clean': './data/original/pc/train_clean.txt',
    'sst2': './data/original/sst2/train.txt',
    'subj': './data/original/subj/train.txt',
    'trec': './data/original/trec/train.txt',
    'kaggle_med': './data/original/kaggle_med/train.txt',
    'yelp': './data/original/yelp/train.txt',
    'bbc': './data/original/bbc/train.txt'
}    
for dataset_name in dict_datasets:
    path = dict_datasets[dataset_name]
    data = pd.read_csv(path, delimiter='\t', header=None, names=['class', 'text'])
    print(f'Loaded {len(data)} rows of data from {dataset_name}')

    # Take only one sample from each dataset
    data = data.sample(n=1)
    print(data)
    print('=====================================')

In [ ]:
data = pd.read_csv('./data/original/pc/train.txt', delimiter='\t', header=None, names=['class', 'text'])

In [ ]:
data = data.sample(n=1)
print(data)